# Aprendizaje por refuerzo

Instalar las siguientes librerias para usar esta notebook:

!pip install swig

!pip install cmake pygame gym[all]

!pip install stable_baselines3

Referencia: https://www.gocoder.one/blog/rl-tutorial-with-openai-gym

![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/refuerzo.png)
Fuente de la imagen: medium.com

![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/RLdog.jpg)
Fuente: kdnuggets.com

Objetivo: maximizar la recompenza.

**Definiciones importantes:**

- Agente: modelo a entrenar para que tome decisiones.
- Ambiente: Entorno donde interactúa el agente. El ambiente limita y pone reglas.

Entre ambiente y agente hay las siguientes relaciones:

- Acción: posibles acciones que puede tomar el agente en determinado momento.
- Estado (del ambiente): indicadores del ambiente sobre cómo están los diversos elementos que lo componen en determinado momento.
- Recompensas (o penalización): cada acción del agente obtiene un premio o una penalización.

Ver: https://towardsdatascience.com/hands-on-introduction-to-reinforcement-learning-in-python-da07f7aaca88


![](https://raw.githubusercontent.com/igomezv/DataScienceIntermedio/main/img/RLalg.png)
Fuente: medium.com

#### Usar el entorno TAXI

**Contexto:** Taxi es un entorno disponible en Open-IA gym (https://www.gymlibrary.dev/environments/toy_text/taxi/).

- El objetivo de Taxi es recoger pasajeros y dejarlos en el destino en la menor cantidad de movimientos.

- Se comparara un agente de taxis que realiza acciones al azar con uno entrenado mediante Aprendizaje Reforzado.

- Acciones: moverse hacia arriba, abajo, izquierda, derecha, recoger o dejar pasajeros.

Descripción de la documentación:

Hay cuatro ubicaciones designadas en el mundo de la cuadrícula indicadas por R (rojo), G (verde), Y (amarillo) y B (azul). Cuando el episodio inicia, el taxi parte de una casilla aleatoria y el pasajero se encuentra en una ubicación aleatoria. El taxi conduce hasta la ubicación del pasajero, lo recoge, conduce hasta el destino del pasajero (otra de las cuatro ubicaciones especificadas) y luego deja al pasajero. Una vez que se deja al pasajero, el episodio termina.

Map:

+---------+

|R: | : :G|

| : | : : |

| : : : : |

| | : | : |

|Y| : |B: |

+---------+

##### Comportamiento


Hay 6 acciones deterministas discretas:

    0: mover al sur

    1: mover al norte

    2: moverse hacia el este

    3: muévete hacia el oeste

    4: pasajero de recogida

    5: dejar al pasajero

Observaciones

Hay 500 estados discretos ya que hay 25 posiciones de taxi, 5 ubicaciones posibles del pasajero (incluido el caso cuando el pasajero está en el taxi) y 4 ubicaciones de destino.

Tenga en cuenta que hay 400 estados a los que se puede llegar durante un episodio. Los estados faltantes corresponden a situaciones en las que el pasajero se encuentra en el mismo lugar que su destino, ya que esto suele indicar el final de un episodio. Se pueden observar cuatro estados adicionales justo después de un episodio exitoso, cuando tanto el pasajero como el taxi están en el destino. Esto da un total de 404 estados discretos alcanzables.

Cada espacio de estado está representado por la tupla: (fila_taxi, col_taxi, ubicación_pasajero, destino)


Ubicaciones de pasajeros:

    0: R (rojo)

    1: G (verde)

    2: Y (amarillo)

    3: B (azul)

    4: en taxi

Destinos:

    0: R (rojo)

    1: G (verde)

    2: Y (amarillo)

    3: B (azul)

Recompensas

     -1 por paso a menos que se active otra recompensa.

     +20 entregando pasajero.

     -10 ejecutar acciones de “recogida” y “devolución” de forma ilegal.
     
     - Una ilegalidad consiste en ejecutar las acciones"recoger"o "devolver" en lugares donde no hay personas.

![](https://www.gocoder.one/static/state-rewards-62ab43a53e07062b531b3199a8bab5b3.png)

Como llegar al punto R sin perder tantos puntos?

## Q-Learning

### Tablas-Q

Una tabla Q es una tabla de búsqueda que almacena valores que representan las recompensas futuras máximas esperadas que el agente puede esperar por una acción determinada en un estado determinado (conocidos como valores Q).

![](https://i.stack.imgur.com/Bn6MY.gif)

Estas tablas se suelen inicializar en 0s

qtable = np.zeros((state_size, action_size))


Hyperparámetros:

- Tasa de aprendizaje (α): qué tanto el agente debe aceptar nueva información sobre la información previamente aprendida

- Factor de descuento (γ): qué tanto el agente debe considerar las recompensas que podría recibir en el futuro frente a su recompensa inmediata.

#### Primero, analicemos el comportamiento aleatorio

In [ ]:
!pip install cmake pygame gym[all]
!pip install stable_baselines3

  Using cached swig-4.1.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.8 MB)
  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Preparing metadata (setup.py) ... done
  Using cached mujoco_py-2.1.2.14-py3-none-any.whl (2.4 MB)
  Using cached lz4-4.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached mujoco-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Using cached pygame-2.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Using cached pytest-7.0.1-py3-none-any.whl (296 kB)
  Using cached ale_py-0.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.6 MB)
  Using cached glfw-2.6.2-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-manylinux2014_x86_64.whl (208 kB)
  Using cached py-1.11.0-py2.py3-none-any.whl (98 kB)
  Using cached fasteners-0.19-py3-none-any.whl (18 kB)


In [ ]:
import gym
import matplotlib.pyplot as plt

In [ ]:
env = gym.make('Taxi-v3')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# create a new instance of taxi, and get the initial state
state = env.reset()

In [ ]:
import gym
import numpy as np
import random

def randomtaxi():
    total_reward = 0
    # crear entorno de Taxi
    env = gym.make('Taxi-v3')

    # crear una instancia de Taxi, y obtener un estado inicial
    state = env.reset()

    num_steps = 9

    for s in range(num_steps+1):
        print("Paso: {}/{}".format(s+1, num_steps+1))

        # muestrear una accion aleatoria de la lista de acciones posibles
        action = env.action_space.sample()
        # ejecutar esta accion sobre el entorno
        new_state, reward, done, info = env.step(action)
        # Update to our new state
        state = new_state
        total_reward += reward
        print("Accion {} Estado {} Recompensa {} Total rec {}".format(action, state, reward, total_reward))


        # imprimir nuevo estado
        env.render()

        # if done, finish episode
        if done == True:
            break


    # terminar y cerrar el entorno del taxi
    env.close()
    return total_reward

In [ ]:
randomtaxi()

Paso: 1/10
Accion 4 Estado 96 Recompensa -1 Total rec -1
Paso: 2/10
Accion 3 Estado 76 Recompensa -1 Total rec -2
Paso: 3/10
Accion 2 Estado 96 Recompensa -1 Total rec -3
Paso: 4/10
Accion 3 Estado 76 Recompensa -1 Total rec -4
Paso: 5/10
Accion 0 Estado 176 Recompensa -1 Total rec -5
Paso: 6/10
Accion 1 Estado 76 Recompensa -1 Total rec -6
Paso: 7/10
Accion 0 Estado 176 Recompensa -1 Total rec -7
Paso: 8/10
Accion 2 Estado 196 Recompensa -1 Total rec -8
Paso: 9/10
Accion 0 Estado 296 Recompensa -1 Total rec -9
Paso: 10/10
Accion 5 Estado 296 Recompensa -10 Total rec -19


-19

In [ ]:
import numpy as np
import gym
import random

def rltaxi():

    # create Taxi environment
    env = gym.make('Taxi-v3')

    # initialize q-table
    state_size = env.observation_space.n
    action_size = env.action_space.n
    qtable = np.zeros((state_size, action_size))

    # hyperparameters
    learning_rate = 0.9
    discount_rate = 0.8
    epsilon = 1.0
    decay_rate= 0.005

    # training variables
    num_episodes = 9
    max_steps = 99 # per episode

    # training
    for episode in range(num_episodes):

        # reset the environment
        state = env.reset()
        done = False

        for s in range(max_steps):

            # exploration-exploitation tradeoff
            if random.uniform(0,1) < epsilon:
                # explore
                action = env.action_space.sample()
            else:
                # exploit
                action = np.argmax(qtable[state,:])

            # take action and observe reward
            new_state, reward, done, info = env.step(action)

            # Q-learning algorithm
            qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

            # Update to our new state
            state = new_state

            # if done, finish episode
            if done == True:
                break

        # Decrease epsilon
        epsilon = np.exp(-decay_rate*episode)

    print(f"Training completed over {num_episodes} episodes")
    input("Press Enter to watch trained agent...")

    # watch trained agent
    state = env.reset()
    done = False
    total_reward = 0

    for s in range(max_steps+1):

        print(f"TRAINED AGENT")
        print("Paso {}".format(s+1))

        action = np.argmax(qtable[state,:])
        new_state, reward, done, info = env.step(action)
        total_reward += reward
        env.render()

        total_reward += reward
        print("Accion {} Estado {} Recompensa {} Total rec {}".format(action, state, reward, total_reward))
        state = new_state
#comentar esto y volver a correr
        if done == True:
          break

    env.close()


In [ ]:
rltaxi()

Training completed over 9 episodes
Press Enter to watch trained agent...
TRAINED AGENT
Paso 1
Accion 0 Estado 381 Recompensa -1 Total rec -2
TRAINED AGENT
Paso 2
Accion 0 Estado 481 Recompensa -1 Total rec -4
TRAINED AGENT
Paso 3
Accion 0 Estado 481 Recompensa -1 Total rec -6
TRAINED AGENT
Paso 4
Accion 0 Estado 481 Recompensa -1 Total rec -8
TRAINED AGENT
Paso 5
Accion 0 Estado 481 Recompensa -1 Total rec -10
TRAINED AGENT
Paso 6
Accion 0 Estado 481 Recompensa -1 Total rec -12
TRAINED AGENT
Paso 7
Accion 0 Estado 481 Recompensa -1 Total rec -14
TRAINED AGENT
Paso 8
Accion 0 Estado 481 Recompensa -1 Total rec -16
TRAINED AGENT
Paso 9
Accion 0 Estado 481 Recompensa -1 Total rec -18
TRAINED AGENT
Paso 10
Accion 0 Estado 481 Recompensa -1 Total rec -20
TRAINED AGENT
Paso 11
Accion 0 Estado 481 Recompensa -1 Total rec -22
TRAINED AGENT
Paso 12
Accion 0 Estado 481 Recompensa -1 Total rec -24
TRAINED AGENT
Paso 13
Accion 0 Estado 481 Recompensa -1 Total rec -26
TRAINED AGENT
Paso 14
Accion 0

###### Ejercicio comparar recompensas de ambos metodos con 3 diferentes numeros de iteraciones.